# Remote gRPC agents - host

This notebook is a host for the remote agents client in the `remote_grpc_client` notebook.

Please refer to the `remote` notebook for further details about remote agents.

**NOTES**
- Run this notebook `remote_grpc_client` notebook AFTER running this one
- Remember to run last cell `host.stop()` to stop the server and release the port

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow, Conversation
from vanilla_aiagents.llm import AzureOpenAILLM
from vanilla_aiagents.remote.remote import RemoteAskable
from vanilla_aiagents.remote.grpc import GRPCConnection
from dotenv import load_dotenv

load_dotenv(override=True)

In [3]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.remote.grpc").setLevel(logging.DEBUG)

In [ ]:
agent1 = Agent(id="agent1", llm=llm, description="Call this agent for general purpose questions", system_message = """You are an AI assistant
    Your task is to help the user with their questions.
    Always respond with the best answer you can generate.
    If you don't know the answer, respond with "I don't know".
    Always be polite and helpful.
    """)

In [5]:
from vanilla_aiagents.remote.remote import RemoteAskable
from vanilla_aiagents.remote.grpc import GRPCHost, GRPCConnection

# Values here must match the client's values
host = GRPCHost(askables=[agent1], host="localhost", port=5002)


In [ ]:
host.start()

In [ ]:
connection = GRPCConnection(url="localhost:5002")
remote = RemoteAskable(id="agent1", connection=connection)
flow = Workflow(askable=remote)

In [ ]:
flow.restart()
# flow.run("Which is the capital of France?")
# flow.conversation.messages

for message in flow.run_stream("Which is the capital of France?"):
    print(message)

In [9]:
# IMPORTANT: Stop the host to release the port
host.stop()